In [1]:
#defining imports
from PIL import Image
import os
import numpy as np
from tempfile import mkdtemp
import os.path as path
import matplotlib.pyplot as plt
import numpy as np
import theano
import keras
import traceback
from keras.models import Sequential
from scipy.misc import imread, imresize, imsave
from keras.layers.core import Flatten, Dense, Dropout,Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D,UpSampling2D,UpSampling1D,Cropping2D
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical
%matplotlib inline
from keras import backend as K
from keras.utils import np_utils
K.set_image_dim_ordering('th')
import traceback
from scipy import ndimage
from sklearn.cross_validation import train_test_split
from keras import callbacks
remote = callbacks.RemoteMonitor(root='http://172.21.10.13:9000')

In [2]:
def imageResize(basename,imageName):
    """
    resize images
    basename : eg. /home/sunil/fishes/bet
    iamgename : xyz.jpg
    """
    new_width  = 224
    new_height = 224
    try:  
        img = Image.open(basename+"/"+imageName) # image extension *.png,*.jpg
        img = img.resize((new_width, new_height), Image.ANTIALIAS)
        img.save(basename+'_resized/'+imageName)
    except:
        os.mkdir(basename+'_resized/')
        img = Image.open(basename+"/"+imageName) # image extension *.png,*.jpg
        img = img.resize((new_width, new_height), Image.ANTIALIAS)
        img.save(basename+'_resized/'+imageName)

In [3]:
def resizer(folderPath):
    """
    to resize all files present in a folder
    resizer('/home/sunil/imageTagging/data/allCats_resized/')
    resizer('/home/sunil/imageTagging/data/allCats_resized/')
    """
    
    for subdir, dirs, files in os.walk(folderPath):
        for fileName in files:
            try:
        #         print os.path.join(subdir, file)
                filepath = subdir + os.sep + fileName
                if filepath.endswith(".jpg"):
                    imageResize(subdir,fileName)
            except:
                print traceback.print_exc()
                os.remove(subdir+"/"+fileName)

In [4]:
# resizer('/home/sunil/imageTagging/downloadData/SAFE')
# resizer('/home/sunil/imageTagging/downloadData/UNSAFE')

In [5]:
def load_image( infilename ) :
    """
    loads file from disk to ram as numpy array
    :param infilename: 
    :return: 
    """
    img = ndimage.imread( infilename )
    data = np.asarray( img, dtype="int32" )
    resized = data.reshape(data.shape[2],data.shape[0],data.shape[1])
    return resized

In [6]:
def turnToNumpy(folderPath):
    """
    turn stored images on disk to numpy
    turnToNumpy('/home/sunil/imageTagging/def generateResponse(class1Size,class2Size):
    y = [] 
    for i in range(class1Size):
        y.append(0)
    for i in range(class2Size):
        y.append(1)
    return ydata/allCats')
    turnToNumpy('/home/sunil/imageTagging/data/allDogs')
    """
    temp = []
    for subdir, dirs, files in os.walk(folderPath):
        for fileName in files:
                try:
                    filepath = subdir + os.sep + fileName
                    if load_image(subdir+"/"+fileName).shape == (3,224,224):
#                         print filepath
                        temp.append(load_image(subdir+"/"+fileName))
                except:
                    os.remove(subdir+"/"+fileName)
        
    return np.asarray(temp)

In [7]:
# loading safe and unsafe images
SAFE =  turnToNumpy('/home/sunil/imageTagging/downloadData/SAFE_resized')
UNSAFE =  turnToNumpy('/home/sunil/imageTagging/downloadData/UNSAFE_resized')

In [8]:
# printing shape of both dataset
print SAFE.shape
print UNSAFE.shape

(5805, 3, 224, 224)
(8081, 3, 224, 224)


In [9]:
def generateResponse(class1Size,class2Size):
    """
    generate response column/ label column
    if  fag]fe images then 1 else 0 for unsafe
    :param class1Size: 
    :param class2Size: 
    :return: 
    """
    y = [] 
    for i in range(class1Size):
        y.append(0)
    for i in range(class2Size):
        y.append(1)
    return y

In [10]:
X=[]
y=[]

# creating array of data
X.extend(SAFE)
X.extend(UNSAFE[:len(SAFE)])
# generating response
y = generateResponse(len(SAFE),len(SAFE))

# CHECKING SHAPE OF DATA AND LABELS
print (len(X))
print (len(y))

# CONVERTING DATA AND LABELS TO NUMPY ARRAY
X = np.asarray(X,dtype="float32")
y = np.asarray(y,dtype="int8")

# USING categorical_crossentropy AS LOSS FUNCTION, IT REQUIRE ONE HOT VECTOR FOR RESPONSE
# SO CONVERTING TO CATEGORICAL
y = to_categorical(np.asarray(y))

# PRINTING SHAPE OF NUMPY ARRAY
print (X.shape)
print (y.shape)

11610
11610
(11610, 3, 224, 224)
(11610, 2)


In [11]:
# NORMALIZING IMAGE FROM 0 TO 1 BY DEVIDING IMAGE WITH 255.

X = X/255.0

[ 1.  0.]
(11610, 2)


In [12]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=13)
print ("x_train : ", X_train.shape,"x_test : ",X_test.shape,"y_train : ",y_train.shape,"y_test : ",y_test.shape)

('x_train : ', (8127, 3, 224, 224), 'x_test : ', (3483, 3, 224, 224), 'y_train : ', (8127, 2), 'y_test : ', (3483, 2))


In [13]:
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
# model.summary()

In [ ]:
# sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer='sgd', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=10, nb_epoch=10,
          verbose=1, validation_data=(X_test, y_test),callbacks=[remote])

In [14]:
model.load_weights("NSFW_wholeNet_10.h5")

In [15]:
pr = model.predict_classes(X_test)

3483/3483 [==============================] - 50s    


In [16]:
actual =  [] 
for i in y_test:
    if(i[0]==1):
        actual.append(0)
    else:
        actual.append(1)

In [1]:
imageNumber = 0
for imageNumber in range (0,len(X_test)):
    tempX = X_test[imageNumber].reshape(X_test[imageNumber].shape[1],X_test[imageNumber].shape[2],X_test[imageNumber].shape[0])
#     plt.imshow(tempX, interpolation='nearest')
#     plt.show()
    predicted = pr[imageNumber]
#     print predicted
#     print actual[imageNumber]
    if (int(actual[imageNumber]) == 1 and int(predicted)== 1):
        imsave("tp/"+str(imageNumber)+"_"+str(actual[imageNumber])+"_"+str(predicted)+".png",tempX)
    if (int(actual[imageNumber]) == 0  and int(predicted)== 0):
        imsave("tn/"+str(imageNumber)+"_"+str(actual[imageNumber])+"_"+str(predicted)+".png",tempX)
    if (int(actual[imageNumber]) == 0  and int(predicted)== 1):
        imsave("fp/"+str(imageNumber)+"_"+str(actual[imageNumber])+"_"+str(predicted)+".png",tempX)
    if (int(actual[imageNumber]) == 1  and int(predicted)== 0):
        imsave("fn/"+str(imageNumber)+"_"+str(actual[imageNumber])+"_"+str(predicted)+".png",tempX)

In [2]:
print (len(actual))
print (len(pr))
fp=0
fn=0
tp=0
tn=0
for no in range(0,len(actual)):
    if actual[no] == 1 and pr[no]==1:
        tp+=1
    elif actual[no] == 0 and pr[no]==0:
        tn += 1
    elif actual[no] == 1 and pr[no]==0:
        fn += 1
    elif actual[no] == 0 and pr[no]==1:
        fp += 1
    
print (fp,fn,tp,tn,"Accuracy : ",float(tp+tn)/float(tn+tp+fp+fn))